In [1]:
import requests
import pandas as pd
import matplotlib 
import numpy as np
import pprint as pp
import csv

In [2]:
### Testing output of Discovery/movie url api method
url = "https://api.themoviedb.org/3/discover/movie?api_key=1058f1538710de09e2cc162ffc042010&primary_release_date.gte=2000&with_watch_providers=8|118|21&watch_region=AU&sort_by=primary_release_date.asc&with_original_language=en"

response = requests.get(url).json()
tmdb_df2 = pd.DataFrame(response["results"])
response
# Outdated - since 118 -> 119 (HBO was removed for Amazon Prime)


{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': None,
   'genre_ids': [18, 878, 53],
   'id': 65397,
   'original_language': 'en',
   'original_title': 'Nightfall',
   'overview': 'The people of the planet Aeon, blessed with six suns, have never experienced night. When an archeological excavation uncovers evidence of an ancient catastrophe, all signs point towards the impending darkness of the very first Nightfall. Panic erupts as the suns slowly disappear one by one. Science struggles against superstition as the people race to comprehend the approaching Nightfall... the end of the world as they know it. Based on Isaac Asimov\'s classic story "Nightfall," voted "The Greatest Science-Fiction Story OF All Time."',
   'popularity': 1.555,
   'poster_path': '/l6fGkWUa48j5BltRKWY1ipXR3gc.jpg',
   'release_date': '2000-01-01',
   'title': 'Nightfall',
   'video': False,
   'vote_average': 3.3,
   'vote_count': 4},
  {'adult': False,
   'backdrop_path': '/iwwZsNXfRajJ1HLVQ5xM4Ic

In [3]:
### Cleaning up url with a filter string as params
url = "https://api.themoviedb.org/3/discover/movie?api_key=1058f1538710de09e2cc162ffc042010"
filter_string = {
    "primary_release_date.gte":"2000",
    "with_watch_providers":"8|119|21",
    "watch_region":"AU",
    "sort_by":"primary_release_date.asc",
    "with_original_language":"en"
    }
response = requests.get(url,params=filter_string).json()
tmdb_df = pd.DataFrame(response["results"])
response

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': None,
   'genre_ids': [99],
   'id': 1011182,
   'original_language': 'en',
   'original_title': 'Close Encounters: Proof of Alien Contact',
   'overview': 'This documentary examines whether UFOs are real. It examines over 250 amateur video clips, evaluates several recent revelations about Roswell, and examines footage of the removal of an alien implant.',
   'popularity': 0.6,
   'poster_path': '/pZF2CqHfdC59DB6p6GodEknhOhl.jpg',
   'release_date': '2000-01-01',
   'title': 'Close Encounters: Proof of Alien Contact',
   'video': False,
   'vote_average': 5,
   'vote_count': 1},
  {'adult': False,
   'backdrop_path': None,
   'genre_ids': [],
   'id': 923893,
   'original_language': 'en',
   'original_title': "Brad Pitt & Jennifer Aniston: Hollywood's Couple of the Year",
   'overview': 'This documentary explores the sequence of events that led A-listers Brad Pitt and Jennifer Aniston to one another, and recounts their relat

In [4]:
### Save extracted json results into separate lists for later use
url = "https://api.themoviedb.org/3/discover/movie"

adult = []
genre_ids = []
id = []
original_language = []
original_title = []
overview = []
popularity = []
release_date = []
title = []
video = []
vote_average = []
vote_count = []

#(1,501,1)

pages = np.arange(1,373,1) # Updated pages
page = 1
for page in pages:

    filter_string = {
        "api_key":"1058f1538710de09e2cc162ffc042010",
        "primary_release_date.gte":"2000",
        "with_watch_providers":"8|119|21",
        "watch_region":"AU",
        "sort_by":"primary_release_date.asc",
        "with_original_language":"en",
        "page":page

        }
    response = requests.get(url,params=filter_string).json()

    for x in response["results"]:

        adult.append(x["adult"])
        genre_ids.append(x["genre_ids"])
        id.append(x["id"])
        original_language.append(x["original_language"])
        original_title.append(x["original_title"])
        overview.append(x["overview"])
        popularity.append(x["popularity"])
        release_date.append(x["release_date"])
        title.append(x["title"])
        video.append(x["video"])
        vote_average.append(x["vote_average"])
        vote_count.append(x["vote_count"])


In [33]:
### Acquire service providers as a separate column for final dataframe - requires another API call with /watch/providers method
# Save the resulting results as a dictionary of lists, with id = keys, providers = values (could have more than 1 provider)
movie_providers = {}
name = []

filter_string = {
    "api_key":"1058f1538710de09e2cc162ffc042010",
    }

for ids in id:
    print(f"Checking movie id: {ids}")
    url = "https://api.themoviedb.org/3/movie/" + str(ids) + "/watch/providers"
    print(requests.get(url,params=filter_string))
    provider_response = requests.get(url,params=filter_string).json()
    for item in provider_response["results"]["AU"]["flatrate"]:
        if item["provider_name"] in ["Stan","Netflix","Amazon Prime Video"]:
            name.append(item["provider_name"])
            print(name)
            print(f"Movie ID: {ids} successfully extracted.")
    print(f"Currently on: {ids}")
    movie_providers.update({ids:name})
    name = []

print(movie_providers)

TypeError: 'builtin_function_or_method' object is not iterable

In [19]:
### Creating a movies provider dataframe -> saving it as csv file
movie_id = list(movie_providers.keys())
provider_df = pd.DataFrame({
    "id":movie_id,
    "providers":list(movie_providers.values())})
provider_df.to_csv("movie_provider.csv",index=False)

In [8]:
### Acquiring production countries information
iso_country = []
country_name = []
budget = [] # $ USD I believe
revenue = []
runtime = [] # in minutes

filter_string = {
    "api_key":"1058f1538710de09e2cc162ffc042010",
    }

for i, ids in enumerate(id):
    print(f"Processing movie index {i + 1}, movie ID: {ids}")
    url = "https://api.themoviedb.org/3/movie/" + str(ids)
    print(url)
    print(requests.get(url,params=filter_string))
    movie_response = requests.get(url,params=filter_string).json()
    try: 
        iso_country.append(movie_response["production_countries"][0]["iso_3166_1"])
        country_name.append(movie_response["production_countries"][0]["name"])
        budget.append(movie_response["budget"])
        revenue.append(movie_response["revenue"])
        runtime.append(movie_response["runtime"])
    except Exception:
        iso_country.append("Null")
        country_name.append("Null")
        budget.append("Null")
        revenue.append("Null")
        runtime.append("Null")


Processing movie index 1, movie ID: 1011182
https://api.themoviedb.org/3/movie/1011182
<Response [200]>
Processing movie index 2, movie ID: 923893
https://api.themoviedb.org/3/movie/923893
<Response [200]>
Processing movie index 3, movie ID: 245955
https://api.themoviedb.org/3/movie/245955
<Response [200]>
Processing movie index 4, movie ID: 134803
https://api.themoviedb.org/3/movie/134803
<Response [200]>
Processing movie index 5, movie ID: 65397
https://api.themoviedb.org/3/movie/65397
<Response [200]>
Processing movie index 6, movie ID: 44388
https://api.themoviedb.org/3/movie/44388
<Response [200]>
Processing movie index 7, movie ID: 656436
https://api.themoviedb.org/3/movie/656436
<Response [200]>
Processing movie index 8, movie ID: 88007
https://api.themoviedb.org/3/movie/88007
<Response [200]>
Processing movie index 9, movie ID: 10384
https://api.themoviedb.org/3/movie/10384
<Response [200]>
Processing movie index 10, movie ID: 252687
https://api.themoviedb.org/3/movie/252687
<R

In [31]:
url = "https://api.themoviedb.org/3/movie/961651"

filter_string = {
    "api_key":"1058f1538710de09e2cc162ffc042010",
    }

movie_response2 = requests.get(url,params=filter_string).json()
movie_response2

{'adult': False,
 'backdrop_path': '/tObVe1R7JnO3VXBvAOWTEPmmegu.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 99, 'name': 'Documentary'}],
 'homepage': '',
 'id': 961651,
 'imdb_id': 'tt20425026',
 'original_language': 'en',
 'original_title': 'Spider-Man: All Roads Lead to No Way Home',
 'overview': 'Join our hosts JB Smoove and Martin Starr for a celebration of 20 years of Spider-Man™ movies!  From the original Sam Raimi trilogy to Marc Webb’s “amazing” movies to the latest trio from Director Jon Watts, we will take viewers through the stars, the stunts and action, the villains and heroes and an homage to Stan Lee, along with a few surprises. Find your favourite Spider-Man pajamas and prepare to swing through the past two decades of Spider-Man at the movies!',
 'popularity': 33.842,
 'poster_path': '/e2LLmI5wKIrWGAy9Of3yyWu7Szn.jpg',
 'production_companies': [{'id': 5388,
   'logo_path': '/i6tbNeVEi7s1uN97s2o0LhEMuF0.png',
   'name': 'Sony Pictures Home Ente

In [9]:
country_money_df = pd.DataFrame({
    "id":id,
    "iso_country":iso_country,
    "country_name":country_name,
    "budget":budget,
    "revenue":revenue,
    "runtime":runtime
})

country_money_df.to_csv("country_money.csv",index=False)

In [7]:
movie_df = pd.DataFrame({
    "id":id,
    "original_language":original_language,
    "original_title":original_title,
    "genre_ids":genre_ids,
    "release_date":release_date,
    "video":video,
    "adult":adult,
    "popularity":popularity,
    "vote_average":vote_average,
    "vote_count":vote_count})
movie_df.head()

,id,original_language,original_title,genre_ids,release_date,video,adult,popularity,vote_average,vote_count
0,1011182,en,Close Encounters: Proof of Alien Contact,[99],2000-01-01,False,False,0.600,5.0,1
1,923893,en,Brad Pitt & Jennifer Aniston: Hollywood's Coup...,[],2000-01-01,False,False,0.655,0.0,0
2,245955,en,The Amish: A People of Preservation,[],2000-01-01,False,False,0.600,0.0,0
3,134803,en,The Newcomers,"[18, 10751]",2000-01-01,False,False,2.339,3.9,9
4,65397,en,Nightfall,"[18, 878, 53]",2000-01-01,False,False,1.555,3.3,4


In [19]:
### Creating the final csv file
movie_provider_df = pd.read_csv("movie_provider.csv")
first_merge_df = pd.merge(movie_df,movie_provider_df,on="id",how="outer")
cleaned_df = pd.merge(first_merge_df,country_money_df,on="id",how="outer")
cleaned_df.head()

cleaned_df.to_csv("cleaned_movie.csv",index=False)


In [45]:
### Checking some things:

cleaned_df = pd.read_csv("cleaned_movie.csv")

def cleaning_brackets2(df,column):
    cleaned_df[column] = cleaned_df[column].apply(lambda x: x.strip('[]').replace("'",''))
    return cleaned_df

# cleaned_df['genre_ids'] = cleaned_df['genre_ids'].apply(cleaning_brackets)
cleaned_df = cleaning_brackets2(cleaned_df,"providers")

genre_mapping = {
    "28": "Action",
    "12": "Adventure",
    "16": "Animation",
    "35": "Comedy",
    "80": "Crime",
    "9": "Documentary",
    "18": "Drama",
    "10751": "Family",
    "14": "Fantasy",
    "36": "History",
    "27": "Horror",
    "10402": "Music",
    "9648": "Mystery",
    "10749": "Romance",
    "878": "Science Fiction",
    "10770": "TV Movie",
    "53": "Thriller",
    "10752": "War",
    "37": "Western",
    }


# cleaned_df["genre_ids"] = cleaned_df["genre_ids"].apply(lambda x: x.strip('[]'))

cleaned_df['genre_ids'] = cleaned_df['genre_ids'].apply(lambda x: ''.join(map(str, x)) if x else x).apply(lambda x: x.strip('[]'))
cleaned_df['genre_ids'] = cleaned_df['genre_ids'].replace("","Null")
# cleaned_df['genre_ids'] = cleaned_df['genre_ids'].replace(to_replace=genre_mapping)
# cleaned_df['genre_ids'] = cleaned_df['genre_ids'].apply(lambda x: [genre_mapping.get(i) for i in x])
cleaned_df.head(15)

# print(cleaned_df.dtypes)


,id,original_language,original_title,genre_ids,release_date,video,adult,popularity,vote_average,vote_count,providers,iso_country,country_name,budget,revenue,runtime
0,1011182,en,Close Encounters: Proof of Alien Contact,99,1/01/2000,False,False,0.600,5.0,1,Amazon Prime Video,Null,Null,Null,Null,Null
1,923893,en,Brad Pitt & Jennifer Aniston: Hollywood's Coup...,Null,1/01/2000,False,False,0.655,0.0,0,Amazon Prime Video,Null,Null,Null,Null,Null
2,245955,en,The Amish: A People of Preservation,Null,1/01/2000,False,False,0.600,0.0,0,Amazon Prime Video,Null,Null,Null,Null,Null
3,134803,en,The Newcomers,"18, 10751",1/01/2000,False,False,2.339,3.9,9,Amazon Prime Video,US,United States of America,0,0,90
4,65397,en,Nightfall,"18, 878, 53",1/01/2000,False,False,1.555,3.3,4,Stan,US,United States of America,0,0,85
5,44388,en,It Had to Be You,"35, 10749",1/01/2000,False,False,9.691,6.2,17,Amazon Prime Video,US,United States of America,0,0,90
6,656436,en,Summer in the Suburbs,"80, 18",10/01/2000,False,False,0.600,0.0,0,Amazon Prime Video,Null,Null,Null,Null,Null
7,88007,en,The Extreme Adventures of Super Dave,"28, 35",11/01/2000,False,False,2.948,4.3,14,Amazon Prime Video,Null,Null,Null,Null,Null
8,10384,en,Supernova,"878, 27, 14, 53",14/01/2000,False,False,13.377,5.0,312,Amazon Prime Video,CH,Switzerland,90000000,14828081,91
9,252687,en,Hell's Kitchen: A New York Neighborhood,Null,15/01/2000,False,False,0.600,0.0,0,Amazon Prime Video,Null,Null,Null,Null,Null
